# Plots and Kmeans clustering

# Library import, etc

In [ ]:
# created based on cellular-05-new-weekend-modified-r6-ISD.ipynb

%reset -f 

In [ ]:
import datetime
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

import random
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn import ensemble
from sklearn import linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import sklearn.model_selection as ms

from sklearn.model_selection import GridSearchCV
from sklearn import neighbors

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler 

from sklearn import svm
from sklearn.svm import SVC, SVR

import statsmodels.api as sm 

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

import geopy.distance

from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot

from statsmodels.graphics.tsaplots import plot_acf

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM 
    
from keras import optimizers
from keras import callbacks    
from itertools import cycle 
import re
import pickle

In [ ]:
random_state_ = 108
seed_ = 9
np.random.seed(seed_)  # this is numpy
random.seed(seed_) # this is python module 

print( np.random.get_state()[1][0] ) # lsw) At this moment, seed_ will be printed. But then, seed will keep changing 

drop_col_th = 0.2 # drop columns by this ratio of NA or null 

In [ ]:
now = datetime.datetime.now()
fname_part = now.strftime("%Y%m%d_%H%M%S") 

# functions

# series_to_supervised

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, mode_=1):
    # n_in: Number of lag observations as input (X). Values may be between [1..len(data)] Optional. Defaults to 1.
    # n_out: Number of observations as output (y). Values may be between [0..len(data)-1]. Optional. Defaults to 1.    
    # mode_ = 1 => var, var2,..., else: 'user', 'bytes'....
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data) # convert to a dataframe 
    cols, names = list(), list() # empty list 
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        if mode_ == 1: 
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        else: 
            names += [('%s(t-%d)' % (c_name, i)) for c_name in list(data.columns) ]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if mode_ == 1: 
            if i == 0:
                names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
            else:
                names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
        else: 
            if i == 0:
                names += [('%s(t)' % (c_name)) for c_name in list(data.columns)]
            else:
                names += [('%s(t+%d)' % (c_name, i)) for c_name in list(data.columns)]
            
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def dto_str_wo_yr_sec(dto_in):
    # Given a datetime object as input, output a string w/ year, sec removed
    # input: datetime object
    # output: string
    
    return dto_in.strftime( "%m-%d %H:%M" )

In [ ]:
def Filter(string, substr): 
    import re
    
    return [str for str in string if
             any(sub in str for sub in substr)] 

In [ ]:
def list_rem_red(list_):
    # remove redundancy from a list, while keeping order
    u_list = [] # unique list
    [u_list.append(item) for item in list_ if item not in u_list]
    
    return u_list

In [ ]:
def list_diff(list1, list2):
    out = [item for item in list1 if not item in list2]
    return out

In [ ]:
# https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

In [ ]:
thre = float(input('Enter threshold for num_signchange(0-1):  '))

def num_signchange(x):
    # x is a series 
    
    if x.empty:
        return None
    else: 
        # https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python/21171725
        y = np.array(x)
        y_ = np.array(y[1:]) - np.array(y[0:-1])
        pos1 = np.where(np.diff(np.signbit( y_ )))[0] # find where sign changes occur, not the latter posion but the front position 
        pos2 = np.nonzero( np.abs(y_)> thre )[0]  # assuming minmax scaled, I assumed max 1 x 0.2 as the slope criterion

        return len( np.intersect1d(pos1, pos2) )

# Enter conditions

In [ ]:
###################################################################
# Used for df_traffic_set (subset around t_bs_i), EDA, performance estimation, plot df_traffic_stat_geo
###################################################################
    
adj_bs_choice = 3 

max_n_adj_bs = 7 
max_n_adj_bs_rest = -1 
    
input_choice = 1

wknd_in_input = 2

pred_target_choice = 1

read_df_dist_m = 1 

make_plots_or_not = 1 


Do_EDA_hist_df_traffic = 1 

Do_plot_df_topology = 1 


Do_corr_analy_df_traffic_set = 1 



Do_df_traffic_FE = 1 
Do_df_traffic_inc_ISD = 1 
if Do_df_traffic_inc_ISD == 1 and Do_df_traffic_FE == 1:
    Do_df_traffic_FE_inc_ISD = 1 
if Do_df_traffic_FE == 1: 
    Do_df_traffic_FE_inc_num_signchange = 1 # 

Do_df_traffic_FE2m = 1 

if Do_df_traffic_FE2m == 1: 
    FE2m_ylogplot = 1 # int(input('Enter 1 to make yaxis log scale for FE2m, 2 not to:  '))

DoKmeansCluster = 1 
if DoKmeansCluster  == 1:
    DoKmeansCluster_save = 1 
if DoKmeansCluster in [1,2]: 
    DoKmeansCluster_xy_features = 1 
    
tr_ratio = 0.7 

# parameter input for reframing 
n_in_time = 30 # int(input('Enter n_in_time(>1 for multiple lag input):  ')) # 1 means, (t-1) only, 2 means (t-2) and (t-1)
n_out_time = 10 # int(input('Enter n_out_time:  ')) # 1 means t only, 2 means t and t+1

use_early_stop = 2 
    
Do_DimRedAnaly = 1 

In [ ]:
input_ch_lst = [ ['NULL'], ['users'], ['packets'], ['bytes'], ['users', 'packets'], ['users', 'bytes'], ['packets', 'bytes'], ['users', 'packets', 'bytes'] ]
pred_tg_ch_lst = input_ch_lst.copy()

input_col = 1 # input_ch_lst[input_choice].copy()


pred_target_col = pred_tg_ch_lst[pred_target_choice]

shorten_bs = 1

# df_traffic and df_toplogy are read

In [ ]:
# datetime.datetime.fromtimestamp( int(x) ) => returns ex) datetime.datetime(2012, 8, 19, 1, 0), a datetime object 
# x: numpy.int64 object 
    
def parse03(x):
    return datetime.datetime.fromtimestamp( int(x) )
        # returned example: 2012-08-19 01:00:00
        # Otherwise, directly from the csv file, time_hour column would be like: ex) 1345305600
        # Looks like, int(x) works same as just x 
        # datetime.fromtimestamp: Return the local date and time corresponding to the POSIX timestamp, 
        # such as is returned by...
        # csv file's Time_hour definition: hourly timestamp in UNIX epoch time (time zone GMT+8).

df_traffic = pd.read_csv("city-cellular-traffic-map-master/traceset/cellular_traffic_full.csv", \
                     parse_dates = [1],date_parser = parse03, header=0)

df_topology = pd.read_csv("city-cellular-traffic-map-master/traceset/topology.csv", header = 0, index_col=0)
    # note) longitude, latitude: usually units are in degrees # bs_i works as index    
    
print( df_traffic.describe() )
print('\n')
print( df_topology.describe() )

# df_dist_m, df_adj_bs generated, using df_topology
## of bs x max_n_adj_bs

In [ ]:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
# The answers above are based on the Haversine formula, which assumes the earth is a sphere, 
# which results in errors of up to about 0.5% (according to help(geopy.distance)). 
# Vincenty distance uses more accurate ellipsoidal models such as WGS-84, and is implemented in geopy. 

# Update: 04/2018: Note that Vincenty distance is deprecated since GeoPy version 1.13 - you should use geopy.distance.distance() instea

def my_geopy_dist_m(lat1, long1, lat2, long2):
    # return geopy.distance.vincenty([lat1,long1], [lat2, long2]).m
    return geopy.distance.distance([lat1,long1], [lat2, long2]).m

# https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula
def my_geopy_dist_list2_m(lat1, long1, lat2, long2):
    # lat1, long1, lat2, long2: should be in radians 
    R_km = 6371
    diff_long = np.radians(long2 - long1)
    diff_lat = np.radians(lat2 - lat1)
    a = math.sin(diff_lat/2)**2 + math.cos(np.radians(lat1)) * math.cos(np.radians(lat2)) * math.sin(diff_long/2)**2
    b = 2 * math.asin( np.minimum(1, np.sqrt(a))) 
    d = R_km * b
    return d

In [ ]:
header_ = []
for ii in range(0,max_n_adj_bs-1): # -1, since max_n_adj_bs includes center cell
    header_ = header_ + ['adj_bs_i_' + str(ii)]
    
df_dist_m = pd.read_csv( 'city-cellular-traffic-map-master/traceset/df_dist_m.csv', index_col=0, header = 0)
df_adj_bs = pd.read_csv( 'city-cellular-traffic-map-master/traceset/df_adj_bs.csv', index_col=0, header = 0)
df_dist_m = df_dist_m.iloc[:, 0:max_n_adj_bs-1] # ex) if max_n_adj_bs=7, 0:6 ==> means neighboring 6 cells 

# df_traffic is further processed to incorporate weekend info or ISD info

In [ ]:
# boolean to 0/1 mapping     
# to include ISD      
df_traffic = pd.merge( df_traffic, df_dist_m.mean(axis=1).rename('ISDmean'), how='left', on ='bs' )

# df_info generation for df_traffic 

In [ ]:
n_unique =df_traffic.nunique() 
null_perc = df_traffic.isnull().sum()/df_traffic.shape[0]*100
NA_perc = df_traffic.isnull().sum()/df_traffic.shape[0]*100

df_info = pd.DataFrame({'n_unique': n_unique, 'null_perc': null_perc, 'NA_perc': NA_perc})
df_info["dtyp"]=""
df_info["iotyp"]=""

for r_i in range(0, df_info.shape[0] ):
    if df_info['n_unique'][r_i] >=10: # 10: just chosen 
        df_info['dtyp'][r_i] = 'numeric'
    elif df_info['n_unique'][r_i] ==2: # don't care, whether nominal or ordinal 
        df_info['dtyp'][r_i] = 'categorical'
    else:
        df_info['dtyp'][r_i] = 'nominal' 
        
    ########################################################################################
    #  If any 'nominal sh be re-classified as 'ordinal, make your codes. OW, be default, I declared as nominal 
    ########################################################################################

    df_info["iotyp"][r_i]='inp'  
        

print(df_info)

# EDA: numeric predictors' histogram of df_traffic

In [ ]:
if Do_EDA_hist_df_traffic==1:   
    for c_i in range(0, len(df_traffic.columns) ):
        if (df_info.loc[df_traffic.columns[c_i], 'dtyp']=="numeric" ): # print out, either input or output 
            if df_traffic.columns[c_i] in ['time_hour', 'bs']:
                continue
                
                # imp_avg = traindata[numeric_df.iloc[f_i,0]].mean()
                # traindata[numeric_df.iloc[f_i,0]] = traindata[numeric_df.iloc[f_i,0]].fillna(imp_avg)                
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 4))
            sns.distplot( df_traffic.iloc[:, c_i], fit=norm, ax=axes[0]);
                # https://seaborn.pydata.org/generated/seaborn.distplot.html
                # plot a univariate distribution of observations
                # fit=norm: Plot the distribution with a histogram and maximum likelihood gaussian distribution fit:
                    # seesm to draw the black line

            res = stats.probplot( df_traffic.iloc[:, c_i] , plot=plt)
                # check https://stackoverflow.com/questions/48108582/how-to-interpret-scipy-stats-probplot-results
                # this shows probplot, not pp plot or qq plot
                # Generates a probability plot of sample data against the quantiles of a specified theoretical distribution (the normal distribution by default). probplot optionally calculates a best-fit line for the data and plots the results using Matplotlib or a given plot function.

                # plotobject, optiona, If given, plots the quantiles and least squares fit. 
                # res: (osm, osr) is returned 
                
            fig.suptitle( 'histogram, df_traffic, '  + df_traffic.columns[c_i]  )   

# BS shortening
## df_topology, df_traffic are affected

In [ ]:
if shorten_bs == 1:
    lon_min = df_topology.describe().loc['mean','lon']-0.5*df_topology.describe().loc['std','lon']
    lon_max = df_topology.describe().loc['mean','lon']+0.5*df_topology.describe().loc['std','lon']
    lat_min = df_topology.describe().loc['mean','lat']-0.5*df_topology.describe().loc['std','lat']
    lat_max = df_topology.describe().loc['mean','lat']+0.5*df_topology.describe().loc['std','lat']
    
    print(df_topology.shape)
    
    df_topology = df_topology.loc[(df_topology['lon'] >= lon_min) & (df_topology['lon'] <= lon_max) & \
                    (df_topology['lat'] >= lat_min) & (df_topology['lat'] <= lat_max)]
    
    df_traffic = df_traffic.loc[ df_traffic.bs.isin(list(df_topology.index)) ]    
    
    print(df_topology.shape)
    print('\n')
    print( df_traffic.describe() )
    print('\n')
    print( df_topology.head(3) )
    print('\n')    
    print( df_topology.describe() )

In [ ]:
if shorten_bs == 1:
    print("================ df_traffic.head(3) ================")
    print(df_traffic.head(3))

    print("\n================ df_traffic.shape ================")
    print(df_traffic.shape)

    print("\n================ df_traffic.dtypes ================")
    print(df_traffic.dtypes)

    print("================ df_topology.head(3) ================")
    print(df_topology.head(3))

    print("\n================ df_topology.shape ================")
    print(df_topology.shape)

    print("\n================ df_topology.dtypes ================")
    print(df_topology.dtypes)

# plot of df_topology at bs_set locations

In [ ]:
if Do_plot_df_topology == 1: 
    fig, ax = plt.subplots(figsize = (8,7))
    ax.scatter( df_topology.loc[0:2000,:].lon, df_topology.loc[0:2000,:].lat, zorder=1, alpha= 0.2, c='b', s=20)
        
    # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.scatter.html

    # alpha blending value, between 0 (transparent) and 1 (opaque).
    # s: The marker size in points**2. Default is rcParams['lines.markersize'] ** 2.
    ax.set_title('Plotting Spatial Data')

# min max scaling on df_traffic and save as another name: df_traffic_sc

In [ ]:
NA_perc = df_traffic.isnull().sum()/df_traffic.shape[0]*100
print(NA_perc)

In [ ]:
if Do_df_traffic_FE==1:     
    scaler = MinMaxScaler(feature_range=(0, 1))

    Do_scale_FE = 1 

    df_traffic_sc = df_traffic.copy().dropna(axis=0, how='any') # drop rows         

    if Do_scale_FE==1: 
        col_sc = list_diff( df_traffic.columns , ['bs', 'time_hour'] ) # which columns to scale? Ans) except for 'bs' and 'time_hour'
            
        df_traffic_sc[ col_sc ] = scaler.fit_transform( df_traffic_sc[ col_sc ] )      

In [ ]:
## Check null ratio of df_traffic_sc
null_perc = df_traffic_sc.isnull().sum()/df_traffic_sc.shape[0]*100
NA_perc = df_traffic_sc.isnull().sum()/df_traffic_sc.shape[0]*100

In [ ]:
if Do_df_traffic_FE==1:   
    km_analy_ch_lst = [ ['NULL'], ['u-'], ['p-'], ['b-'], ['u-', 'p-'], ['u-', 'b-'], ['p-', 'b-'], ['u-', 'p-', 'b-'] ]
    km_input_choice = int( input('For KM analysis(to generate df_traffic_FE), Input, 1(users), 2(pkt), 3(bytes), 4(users, pkt), 5(users, bytes), 6(pkt, bytes), 7(all):  ') )
    km_input_col = km_analy_ch_lst[km_input_choice].copy()

    km_input_col

# generate df_traffic_FE using df_traffic_sc, not using melt()

In [ ]:
if Do_df_traffic_FE==1:     
    h_vec_choice = int( input('h_vec_choice, 1: 7 intervals, 2: 4 intervals, 3: 2 intervals ') )

    if h_vec_choice==1: # 7 intervals 
        h_vec = [0, 6, 9, 12, 15, 18, 21, 24]
        h_vec__str = ['0_6', '6_9', '9_12', '12_15', '15_18', '18_21', '21_24']
    elif h_vec_choice==2: # 4 intervals   
        h_vec = [0, 8, 13, 18,24]    
        h_vec_str = ['0_8', '8_13','13_18', '18_24' ]
    elif h_vec_choice==3: # 2 intervals        
        h_vec = [0, 8, 24]
        h_vec_str = ['0_8', '8_24']

    col_ = ['u-mean', 'u-std', 'u-min', 'u-max',\
            'p-mean', 'p-std', 'p-min', 'p-max', \
            'b-mean', 'b-std', 'b-min', 'b-max'] # fullset # Start off from fullsets 
    agg_vec = ['mean', 'std', 'min', 'max'] # aggregaton vector, start from fullset 
    daytyp_vec = ['wknd', 'wkdy'] # day type, weekend or weekday 
    stat_choice = int(input('(In generating df_traffic_FE), \
    Enter 1 for mean only, 2 for std only, 3 for mean and std, \
    4: mean, min, 5: std, min, 6: mean, std, min, 7: mean, max, 8: std, max, 9: mean, std, max,\ \
    10: mean, min, max, 11: std, min, max, 12: mean, std, min, max  '))
    
    if stat_choice == 1:
        Filter_by = ['mean']
    elif stat_choice == 2:
        Filter_by = ['std']        
    elif stat_choice == 3:
        Filter_by = ['mean', 'std']           
    elif stat_choice == 4:
        Filter_by = ['mean', 'min']                   
    elif stat_choice == 5:
        Filter_by = ['std', 'min']                   
    elif stat_choice == 6:
        Filter_by = ['mean', 'std', 'min']                   
    elif stat_choice == 7:
        Filter_by = ['mean','max']                   
    elif stat_choice == 8:
        Filter_by = ['std', 'max']                   
    elif stat_choice == 9:
        Filter_by = ['mean', 'std', 'max']                   
    elif stat_choice == 10:
        Filter_by = ['mean', 'min', 'max']                   
    elif stat_choice == 11:
        Filter_by = ['std', 'min', 'max']                   
    elif stat_choice == 12:
        Filter_by = ['mean', 'std', 'min', 'max']  
                     
    col_ = Filter( col_, Filter_by ) # downselect
    agg_vec = Filter( agg_vec, Filter_by ) # downselect
    col_ = Filter( col_, km_input_col ) # further downselect # u-, p-, b-   

    for daytyp in daytyp_vec: # 2 # means expanding along xaxis 
        if wknd_in_input == 2 and daytyp == daytyp_vec[1]: # if then, doing only once is enough 
            continue 
        
        for ii in range( 0, len(h_vec)-1 ): # 7 # means expanding along xaxis 
            cond1 = (df_traffic_sc['time_hour'].dt.hour>=h_vec[ii] )
            cond2 = (df_traffic_sc['time_hour'].dt.hour< h_vec[ii+1])

            cond3 = True
            cond4 = True
                
            cond_comb = cond1 & cond2 & (cond3 | cond4)
            temp1 = df_traffic_sc[ cond_comb ]

            if km_input_choice==1:
                temp2 = temp1.groupby('bs').agg({'users':agg_vec})
            elif km_input_choice==2:
                temp2 = temp1.groupby('bs').agg({'packets':agg_vec})
            elif km_input_choice==3:
                temp2 = temp1.groupby('bs').agg({'bytes':agg_vec})
            elif km_input_choice==4:
                temp2 = temp1.groupby('bs').agg({'users':agg_vec, 'packets':agg_vec})
            elif km_input_choice==5:
                temp2 = temp1.groupby('bs').agg({'users':agg_vec, 'bytes':agg_vec })
            elif km_input_choice==6:
                temp2 = temp1.groupby('bs').agg({'packets':agg_vec, 'bytes':agg_vec })
            elif km_input_choice==7:
                temp2 = temp1.groupby('bs').agg({'users':agg_vec, 'packets':agg_vec, 'bytes':agg_vec })

            str_temp2 = "suffix_ = ['_tz_%d_%d']*len(col_)"%(h_vec[ii], h_vec[ii+1]) # generate suffix_
            exec(str_temp2)
            print('\n')
            print(suffix_) # ex) ['_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy']
            temp2.columns = [m+n for m,n in zip(col_,suffix_)]

            if ii==0 and daytyp == 'wknd': # if occurring for the first time 
                df_traffic_FE = temp2 # FE: feature engineering 
            else: 
                df_traffic_FE = pd.concat( [df_traffic_FE, temp2], axis = 1 )   
                
    if Do_df_traffic_FE_inc_num_signchange == 1: 
        for daytyp in daytyp_vec: # 2 # means expanding along xaxis 
            
            if wknd_in_input == 2 and daytyp == daytyp_vec[1]: # if then, doing only once is enough 
                continue
            
            for ii in range( 0, len(h_vec)-1 ): # 7 # means expanding along xaxis 
                
                cond1 = (df_traffic_sc['time_hour'].dt.hour>=h_vec[ii] )
                cond2 = (df_traffic_sc['time_hour'].dt.hour< h_vec[ii+1])

                cond3 = True
                cond4 = True
                    
                cond_comb = cond1 & cond2 & (cond3 | cond4)
                temp1_1 = df_traffic_sc[ cond_comb ]

                if km_input_choice==1:
                    temp2_1 = temp1_1.groupby('bs').agg({'users':[num_signchange]})
                    col_1_ = ['u-nsignchg'] 
                elif km_input_choice==2:
                    temp2_1 = temp1_1.groupby('bs').agg({'packets':[num_signchange]})
                    col_1_ = ['p-nsignchg'] 
                elif km_input_choice==3:
                    temp2_1 = temp1_1.groupby('bs').agg({'bytes':[num_signchange]})
                    col_1_ = ['b-nsignchg'] 
                elif km_input_choice==4:
                    temp2_1 = temp1_1.groupby('bs').agg({'users':[num_signchange], 'packets':[num_signchange]})
                    col_1_ = ['u-nsignchg', 'p-nsignchg'] 
                elif km_input_choice==5:
                    temp2_1 = temp1_1.groupby('bs').agg({'users':[num_signchange], 'bytes':[num_signchange] })
                    col_1_ = ['u-nsignchg', 'b-nsignchg'] 
                elif km_input_choice==6:
                    temp2_1 = temp1_1.groupby('bs').agg({'packets':[num_signchange], 'bytes':[num_signchange] })
                    col_1_ = ['p-nsignchg', 'b-nsignchg'] 
                elif km_input_choice==7:
                    temp2_1 = temp1_1.groupby('bs').agg({'users':[num_signchange], 'packets':[num_signchange], 'bytes':[num_signchange] })
                    col_1_ = ['u-nsignchg', 'p-nsignchg', 'b-nsignchg'] 

                str_temp2_1 = "suffix_1_= ['_tz_%d_%d']*len(col_)"%(h_vec[ii], h_vec[ii+1]) # generate suffix_    
                exec(str_temp2_1)
                print('\n')
                print(suffix_1_) # ex) ['_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy', '_tz_18_24_wkdy']
                temp2_1.columns = [m+n for m,n in zip( col_1_ ,suffix_1_)]
                
                if Do_scale_FE == 1:
                    temp2_1_sc = pd.DataFrame(data = scaler.fit_transform( temp2_1.values ), index=list(temp2_1.index), columns=list(temp2_1.columns) ) 
                    temp2_1_sc.index.name = 'bs' # give name to the index column
                    # since df_traffic_FE already exists, just keep extending
                    df_traffic_FE = pd.merge( df_traffic_FE, temp2_1_sc, how = 'left', on='bs' )
                else:     
                    # since df_traffic_FE already exists, just keep extending
                    df_traffic_FE = pd.merge( df_traffic_FE, temp2_1, how = 'left', on='bs' )
                
    # this does not need aggregation             
    if Do_df_traffic_FE_inc_ISD == 1:
        if Do_scale_FE == 1:
            temp4 = df_traffic[['bs', 'ISDmean']].drop_duplicates(subset=['bs'], keep='first').set_index('bs')
            temp4_sc = pd.DataFrame(data = scaler.fit_transform( temp4.values ), index=list(temp4.index), columns=list(temp4.columns) ) 
            temp4_sc.index.name = 'bs' # give name to the index column
            df_traffic_FE = pd.merge( df_traffic_FE, \
                temp4_sc, how='left', on ='bs' )
                # dropped duplicates, since having duplicates won't be good 
        else: 
            df_traffic_FE = pd.merge( df_traffic_FE, \
                df_traffic[['bs', 'ISDmean']].drop_duplicates(subset=['bs'], keep='first').set_index('bs'), how='left', on ='bs' )
                    # dropped duplicates, since having duplicates won't be good 
    
    if 1: 
        del temp1, temp2, str_temp2, col_
        if Do_df_traffic_FE_inc_num_signchange == 1: 
            del temp1_1, temp2_1, str_temp2_1, col_1_ 
            if Do_scale_FE == 1:
                del temp2_1_sc
        if Do_df_traffic_FE_inc_ISD == 1:
            del temp4, temp4_sc

In [ ]:
if Do_df_traffic_FE==1:      
    print( df_traffic_FE.head(3) )
    print('\n')
    print( df_traffic_FE.shape )

    n_unique =df_traffic_FE.nunique() 
    null_perc = df_traffic_FE.isnull().sum()/df_traffic_FE.shape[0]*100
    NA_perc = df_traffic_FE.isnull().sum()/df_traffic_FE.shape[0]*100

    print(null_perc)
    print('\n')
    print(NA_perc)
    
    ###############################################
    # For some BS, they do not have enough samples. They may exist only for certain days, for example. 
    # If then, that BS may be fine up to df_traffic_sc, but they may have NaN from the point of df_traffic_FE 
    ###############################################    

# Missing data processing on df_traffic_FE

In [ ]:
if Do_df_traffic_FE==1:  
    df_traffic_FE = df_traffic_FE.dropna(axis = 0, how='any') # drop rows by any 

# df_traffic_FE2m

In [ ]:
if Do_df_traffic_FE2m == 1: 
    for ii in range( 0, len(h_vec)-1 ): # 7
        cond1 = (df_traffic_sc['time_hour'].dt.hour>=h_vec[ii] )
        cond2 = (df_traffic_sc['time_hour'].dt.hour< h_vec[ii+1])
        temp = df_traffic_sc[cond1 & cond2].drop(['time_hour'],1) 
            # temp has: bs	users	packets	bytes	weekend	ISDmean	Hour_info
        
        exec( "Hour_info = ['Hour_%d_%d']"%(h_vec[ii], h_vec[ii+1]) )  # ex: tz_6_9        
        temp['Hour_info'] = Hour_info[0]

        if ii==0:
            df_traffic_FE2 = temp
        else:
            df_traffic_FE2 = pd.concat( [df_traffic_FE2,temp], axis=0 ) # add rows 
            
    print( df_traffic_FE2.head(2) )      
    
    col_list = ['bs', 'Hour_info', 'users', 'packets', 'bytes']
        
    df_traffic_FE2 = df_traffic_FE2[col_list]

    print('/n')
    print( df_traffic_FE2.head(2) )
    del col_list
    

    df_traffic_FE2m = df_traffic_FE2.melt(id_vars=["bs", "Hour_info"], var_name="data_kind", value_name="Value")
        
    df_traffic_FE2m.head(2)

    # https://seaborn.pydata.org/examples/grouped_boxplot.html
    # https://seaborn.pydata.org/examples/grouped_boxplot.html
    sns.set_palette(palette="pastel")
    sns.set_style(style="ticks")

    plt.figure()
    # Draw a nested boxplot to show bills by day and time
    ax = sns.boxplot(x="Hour_info", y="Value", hue="data_kind", palette="colorblind", \
                         data=df_traffic_FE2m, showfliers=False )    

    if FE2m_ylogplot == 1: 
        ax.set(yscale="log")
    ax.set_ylim([1E-8,1])

    sns.despine(offset=10, trim=True)
        # Spines are the lines connecting the axis tick marks and noting the boundaries of the data area. They can be placed at arbitrary positions.2020. 1. 5.
    plt.legend(loc='center right', bbox_to_anchor=(1.20, 0.5), ncol=1)
    # ax.set_title()
    # ax.set(ylim=(0.000000000000000000000000001, 1))
    plt.setp(ax.get_xticklabels(), rotation=45)

    df_traffic_sc.describe()
        
################################################################################
## If data min is 0, then boxplots may not show the minimum value line
################################################################################

# Kmeans clustering #1 (simple), based on df_traffic_FE
### Then, save as a csv file

In [ ]:
if DoKmeansCluster == 1:
    Kmeans_cl_size = int( input('Enter n_clusters for Kmeans clustering: ') )

    # from __future__ import print_function
    %matplotlib inline
        # %matplotlib inline sets the backend of matplotlib to the 'inline' backend: With this backend, the output of plotting commands is displayed inline within frontends like the Jupyter notebook, directly below the code cell that produced it. The resulting plots will then also be stored in the notebook document.2
    
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = 8, 6
    
    kmeans = KMeans()   
        # https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
            # n_clustersint, optional, default: 8, The number of clusters to form as well as the number of centroids to generate.
            # init{‘k-means++’, ‘random’ or an ndarray}, Method for initialization, defaults to ‘k-means++’:
                # ‘k-means++’ : selects initial cluster centers for k-mean clustering in a smart way to speed up convergence. See section Notes in k_init for more details.
                # ‘random’: choose k observations (rows) at random from data for the initial centroids.
            # n_init int, default: 10, Number of time the k-means algorithm will be run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia.
            # max_iterint, default: 300, Maximum number of iterations of the k-means algorithm for a single run.
            # random_state, int, RandomState instance or None (default), Determines random number generation for centroid initialization. Use an int to make the randomness deterministic.
            # algorithm“auto”, “full” or “elkan”, default=”auto”, K-means algorithm to use. 
                # The classical EM-style algorithm is “full”. The “elkan” variation is more efficient by using the triangle inequality, but currently doesn’t support sparse data. 
                # “auto” chooses “elkan” for dense data and “full” for sparse data.

    kmeans.set_params(n_clusters=Kmeans_cl_size, n_init = 30, random_state = random_state_, n_jobs=None)
    kmeans.fit(df_traffic_FE)
    
    if DoKmeansCluster_save == 1: 
        fname = 'kmeans_' + '_hvch_' + str(h_vec_choice) + '_clsz_' + str(Kmeans_cl_size) + '_stch_' + str(stat_choice) + '_' + fname_part + '.pkl'
        pickle.dump(kmeans, open( fname , "wb"))
            # 파이썬에서는 이와 같은 텍스트 이외의 자료형을 파일로 저장하기 위하여 pickle이라는 모듈을 제공한다.
        del fname
    
    df_traffic_FE_cluster = pd.DataFrame({'bs': df_traffic_FE.index, 'cluster':kmeans.labels_ })
    df_traffic_FE_cluster = df_traffic_FE_cluster.set_index('bs')
    df_traffic_FE_cluster.head(2)
    
    if DoKmeansCluster_save == 1: 
        fname = 'df_traffic_FE' + '_hvch_' + str(h_vec_choice) + '_clsz_' + str(Kmeans_cl_size) + \
        '_stch_' + str(stat_choice) + '_' + fname_part + '.csv'
        df_traffic_FE_cluster.to_csv( fname )
        del fname

elif DoKmeansCluster == 2:
    pkl_name = input('Enter pickle file name to load:  ')
    df_traffic_FE_cl_name = int('Enter df_traffic_FE_cluster file name to load:  ')
    
    # https://stackoverflow.com/questions/54879434/how-to-use-the-pickle-to-save-sklearn-model
    kmeans = pickle.load(open(pkl_name, "rb"))
    df_traffic_FE_cluster = pd.read_csv(df_traffic_FE_cl_name, header=0)
    
print('\n ====== kmeans.cluster_centers_ ======')
print(kmeans.cluster_centers_[:, 0:10])
print(kmeans.cluster_centers_.shape)

print('\n kmeans.labels_[0:10]')
print(kmeans.labels_[0:15])
print( pd.Series( kmeans.labels_).value_counts() )

plt.hist( kmeans.labels_, bins = np.arange(Kmeans_cl_size+1) - 0.5 ) # -0.5: for center-aligned bars
    # https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
plt.title('kmeans.labels_histogram, cluster size='+str(Kmeans_cl_size))
plt.xticks( np.arange(Kmeans_cl_size)  ) 
plt.xlabel('cluster index')

# Elbow method

In [ ]:
# We can use a graphical tool, called the elbow method, to estimate  𝑘 . The elbow method plots the inertia (sometimes called distortion) for different values of  𝑘#  

from PlottingFunctions import plot_inertia, plot_silhouette
plot_inertia(kmeans, df_traffic_FE, range(1, 20))

# After Kmeans clustering, show clusters on xy plots, uisng df_traffic_FE

In [ ]:
if DoKmeansCluster in [1,2] and DoKmeansCluster_xy_features ==1: # use df_traffic_FE 
    DoKmeansCluster_xy_choice = 2 # 
    
    # https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib
    from itertools import cycle 
    cycol = cycle('bgrcmk')
    # colors = ['turquoise', 'darkorange', 'darkorange', 'silver', 'steelblue'] # since Kmeans clustering with 5 
    colors = cycle(['blue', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    
    # kmeans_label_names = ['cl_']*Kmeans_cl_size + [str(ii) for ii in range(0, Kmeans_cl_size)]
    kmeans_label_names = [m+n for m,n in zip(['CL_']*Kmeans_cl_size, [str(ii) for ii in range(0, Kmeans_cl_size)])]
    # kmeans_label_names = ['clst 0', 'clst 1', 'clst 2', 'clst 3', 'clst 4'] 
    
    # markers_ = ['o', '+', 'x', '*', 'd']
    markers_ = ['o','s', 'x', 'd', '^']
    markers_cyc =cycle(markers_)
    lw = 2    # linewidth of the marker
    alpha = 0.3 # alpha blending value, between 0 (transparent) and 1 (opaque).
    s = 20 # marker size 
    # cmap = get_cmap( len(markers_) )
    
    for ind1 in range(0, df_traffic_FE.shape[1]):
        for ind2 in range(ind1, df_traffic_FE.shape[1]):
            if ind1 != ind2: 
                    
                if DoKmeansCluster_xy_choice == 1: # then display only mean vs mean 
                    if '-mean' not in df_traffic_FE.columns[ind1] and '-mean' not in df_traffic_FE.columns[ind2]: 
                        continue; 
                    # lsw) since I also have 'ISDmean', skip if neigher has 'mean'
                    
                # if different time intervals, continue 
                if len(re.split('[-_]+', df_traffic_FE.columns[ind1])) > 1 and len(re.split('[-_]+', df_traffic_FE.columns[ind2])) > 1:
                    # If either is of length 1, then just plot
                    # If both are of length > 1, then just time zone sameness
                    if re.split('[-_]+', df_traffic_FE.columns[ind1])[3]!=re.split('[-_]+', df_traffic_FE.columns[ind2])[3]:
                        continue # visit next ind1,ind2, do not plot at this time
                    if re.split('[-_]+', df_traffic_FE.columns[ind1])[4]!=re.split('[-_]+', df_traffic_FE.columns[ind2])[4]: 
                        continue # visit next ind1, ind2, do not plot at this time 
                
                plt.figure()
                ax = plt.gca()
                
                for i, kmeans_label_name in zip( range(0, Kmeans_cl_size), kmeans_label_names ): # i: cluster index
                    plt.scatter(df_traffic_FE.to_numpy()[kmeans.labels_ == i, ind1], \
                                df_traffic_FE.to_numpy()[kmeans.labels_ == i, ind2], \
                                c=next(cycol), s = s, alpha=alpha , lw=lw, label=kmeans_label_name, marker = next(markers_cyc) )

                    # plt.scatter(kmeans.cluster_centers_[i, ind1], kmeans.cluster_centers_[i, ind2], marker=markers_[i], s=1000, c=color )
                        # blocked this line. If categorical values, majority vote is better. 

                plt.legend(loc='best', shadow=False, scatterpoints=1)
                plt.title( 'After Kmeans clustering' )  
                plt.xlabel( df_traffic_FE.columns[ind1] )
                plt.ylabel( df_traffic_FE.columns[ind2] )
                
                # ax.set_xscale('log')
                # ax.set_yscale('log')
                
                plt.show()

# PCA analysis 

In [ ]:
if Do_DimRedAnaly == 1: 
    # copied from Computer_Vision_Lab-Radionom-liv-r31-loop.ipynb
    from sklearn.decomposition import PCA

    pca = PCA()
    pca.set_params(n_components = None, random_state = random_state_ ) # in the beginning, do not restrct by n_compo
    pca.fit( df_traffic_FE )    
    
    n_compo = int(input('Enter n_compo for PCA analysis:  '))

    # plt.plot(range(30), pca.explained_variance_ratio_[0:30])
    plt.plot(pca.explained_variance_ratio_[0:30] )
    plt.xlabel('ith components')
    plt.ylabel('Percentage of Variance')
    plt.title('PCA analysis')
    
    plt.xticks(np.arange(0, 30, step = 1))  # S
    
    plt.show()
    print('\n pca.explained_variance_ratio_ is(n_components=None)[0:30]: ')
    print( pca.explained_variance_ratio_[0:30] )
    print('\n total percentage of variance explainedn_components=None)[0:30]: ')
    print( np.sum(pca.explained_variance_ratio_[0:30])  )  

    pca.set_params( n_components=n_compo, random_state = random_state_ ).fit( df_traffic_FE )
    
    df_traffic_FE_PCA = pd.DataFrame( data = pca.transform(df_traffic_FE), index = df_traffic_FE.index, \
                                     columns = ['component  ' + str(i) for i in range(1,n_compo+1)] ) 

In [ ]:
if DoKmeansCluster in [1,2] and DoKmeansCluster_xy_features ==1 and Do_DimRedAnaly == 1: # use df_traffic_FE 
    # https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib
    from itertools import cycle 
    
    # cycol = cycle('bgrcmk')
    # colors = ['turquoise', 'darkorange', 'darkorange', 'silver', 'steelblue'] # since Kmeans clustering with 5 
    
    # kmeans_label_names = ['cl_']*Kmeans_cl_size + [str(ii) for ii in range(0, Kmeans_cl_size)]
    kmeans_label_names = [m+n for m,n in zip(['CL_']*Kmeans_cl_size, [str(ii) for ii in range(0, Kmeans_cl_size)])]
    # kmeans_label_names = ['clst 0', 'clst 1', 'clst 2', 'clst 3', 'clst 4'] 
    
    # markers_ = ['o', '+', 'x', '*', 'd']

    lw = 2    # linewidth of the marker
    alpha = 0.8 # alpha blending value, between 0 (transparent) and 1 (opaque).
    s = 15 # marker size 
    # cmap = get_cmap( len(markers_) )
    
    for ind1 in range(0, df_traffic_FE_PCA.shape[1]):
        for ind2 in range(ind1, df_traffic_FE_PCA.shape[1]):
            if ind1 != ind2: 
                
                colors = ['blue', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']
                colors_cyc = cycle(colors)

                markers_ = ['o','s', 'x', 'd', '^']
                markers_cyc =cycle(markers_)    
                
                plt.figure()
                ax = plt.gca()
                
                for i, kmeans_label_name in zip( range(0, Kmeans_cl_size), kmeans_label_names ): # i: cluster index
                    plt.scatter( df_traffic_FE_PCA[kmeans.labels_ == i].iloc[:, ind1], \
                                 df_traffic_FE_PCA[kmeans.labels_ == i].iloc[:, ind2], \
                                 c=next(colors_cyc), s = s, alpha=alpha , lw=lw, label=kmeans_label_name, \
                                 marker = next(markers_cyc) )

                    # plt.scatter(kmeans.cluster_centers_[i, ind1], kmeans.cluster_centers_[i, ind2], marker=markers_[i], s=1000, c=color )
                        # blocked this line. If categorical values, majority vote is better. 

                plt.legend(loc='best', shadow=False, scatterpoints=1)
                plt.title( 'Kmeans clustering displayed by PCA of ' + str(df_traffic_FE_PCA.shape[0]) + ' BS' )  
                plt.xlabel( 'PCA ' + str(ind1+1) + 'st component' )
                plt.ylabel( 'PCA ' + str(ind2+1) + 'st component' )
                
                # ax.set_xscale('log')
                # ax.set_yscale('log')
                
                plt.show()

# plot of df_topology at bs_list locations(based on df_traffic_FE or df_traffic_FE_cluster) 
# with Kmeans clustering colors

In [ ]:
plot_text = int(input('Enter 1 to display text, 2 not to:  '))

if Do_plot_df_topology == 1: 
    cycol = cycle('bgrcmk')
    plt.figure( figsize=(12,8) )
    
    # colors = cycle(['cyan', 'tomato', 'dodgerblue', 'lawngreen', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    # colors = cycle(['fuchsia', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    colors = cycle(['blue', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    
    # markers_ = ['o', '+', 'x', '*', 'd']
    markers_ = ['o','s', 'x', 'd', '^']
    markers_cyc =cycle(markers_)    

    # all base stations
    for cl_i in range(0, Kmeans_cl_size): # 0...6
        bs_list = list(df_traffic_FE_cluster[ df_traffic_FE_cluster['cluster']==cl_i ].index)

        plt.scatter( df_topology.loc[ bs_list, : ].lon, df_topology.loc[ bs_list, : ].lat, zorder=1, alpha= 0.7, c=next(colors), s=15, label='CL '+str(cl_i) )
        
        # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.scatter.html
        # alpha blending value, between 0 (transparent) and 1 (opaque).
        # s: The marker size in points**2. Default is rcParams['lines.markersize'] ** 2.
        
    plt.legend(loc='best', shadow=False, scatterpoints=1)
    plt.title( 'Locations of ' + str(len( df_traffic_FE_cluster )) + ' base stations' )  
    plt.xlabel('longitude')
    plt.ylabel('latitude')
    plt.show()        
    
    ######################################################################################    
    ################ specific area only display, all clusters, with texts ################
    ######################################################################################
    
    # colors = cycle(['cyan', 'tomato', 'dodgerblue', 'lawngreen', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    # colors = cycle(['fuchsia', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    colors = cycle(['blue', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
    
    # markers_ = ['o', '+', 'x', '*', 'd']
    markers_ = ['o','s', 'x', 'd', '^']
    markers_cyc =cycle(markers_)     
    
    lon_min_sh = 111.071
    lon_max_sh = 111.079
    lat_min_sh = 13.121
    lat_max_sh = 13.129
            
    
    df_topology_sh = df_topology.loc[(df_topology['lon'] >= lon_min_sh) & (df_topology['lon'] <= lon_max_sh) & \
                    (df_topology['lat'] >= lat_min_sh) & (df_topology['lat'] <= lat_max_sh)]

    lon_d_sh = lon_max_sh-lon_min_sh
    lat_d_sh = lat_max_sh-lat_min_sh 

    lon_d_sh_text = lon_d_sh/240 # to be used for text display
    lat_d_sh_text = lat_d_sh/180 # to be used for text display

    plt.figure(figsize=(20,10))
    num_BS = 0 

    for cl_i in range(0, Kmeans_cl_size): # 0...6
        # generated for each cl_i

        bs_list_sh = set.intersection( set(df_traffic_FE_cluster[ df_traffic_FE_cluster['cluster']==cl_i ].index), set(df_topology_sh.index) )
        
        num_BS +=len(bs_list_sh)

        plt.scatter( df_topology_sh.loc[ bs_list_sh, : ].lon, df_topology_sh.loc[ bs_list_sh, : ].lat, \
                    zorder=1, alpha= 0.9, c=next(colors), marker = next(markers_cyc), s=100, label='CL '+str(cl_i) )

        # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.scatter.html
        # alpha blending value, between 0 (transparent) and 1 (opaque).
        # s: The marker size in points**2. Default is rcParams['lines.markersize'] ** 2.

        if plot_text == 1: 
            for ii, bb, lon_, lat_ in zip( range(0,len(bs_list_sh)) , bs_list_sh, df_topology_sh.loc[bs_list_sh,:].lon, df_topology_sh.loc[bs_list_sh,:].lat ):
                plt.text(lon_+lon_d_sh_text, lat_+lat_d_sh_text, str(cl_i)+','+str(bb), fontsize=11, rotation = 20)      

    # plt.legend(loc='best', shadow=False, scatterpoints=1)
    plt.legend(loc='upper left', shadow=False, scatterpoints=1, bbox_to_anchor=(1.02, 1), fontsize='x-large', \
               numpoints = 15)
        # https://stackoverflow.com/questions/4700614/how-to-put-the-legend-out-of-the-plot
    plt.title( 'Locations of ' + str(num_BS) + ' base stations' )  
    plt.xlabel('longitude')
    plt.ylabel('latitude')

    plt.axis([lon_min_sh-lon_d_sh/20, lon_max_sh+lon_d_sh/20, lat_min_sh-lat_d_sh/20, lat_max_sh+lat_d_sh/20])    

    
    # del bs_list_sh
    # del df_topology_sh

In [ ]:
if DoKmeansCluster == 1:
    df_traffic_FE_cluster_sh = df_traffic_FE_cluster.loc[ set.intersection( set(df_topology_sh.index), set(df_traffic_FE_cluster.index) ) ] # slice out accordding the bs indexes
    # set.intersection: otherwise, due to NA, df_traffic_FE_cluster. has less # of BSs than df_topology_sh.index
    
    if DoKmeansCluster_save == 1: 
        fname = 'df_traffic_FE_sh' + '_hvch_' + str(h_vec_choice) + '_clsz_' + str(Kmeans_cl_size) + \
        '_stch_' + str(stat_choice) + '_' + fname_part + '.csv'   
        df_traffic_FE_cluster_sh.to_csv( fname )
        
        del fname

# PCA analysis display for only certain BSs

In [ ]:
if DoKmeansCluster in [1,2] and DoKmeansCluster_xy_features ==1 and Do_DimRedAnaly == 1: # use df_traffic_FE 
    # https://stackoverflow.com/questions/14720331/how-to-generate-random-colors-in-matplotlib
    
    text_disp = int(input('Enter 1 to display CL and BS index, 2 not to: :  '))
    
    xx_d_text = 2/500 # to be used for text display # I think PCA is -1 ~ 1
    yy_d_text = 2/500 # to be used for text display # I think PCA is -1 ~ 1
    
    alpha = 0.7 # alpha blending value, between 0 (transparent) and 1 (opaque).
    s = 50 # marker size 
    
    for ind1 in range(0, df_traffic_FE_PCA.shape[1]): # x axis
        for ind2 in range(ind1, df_traffic_FE_PCA.shape[1]): # y axis 
            
            colors = cycle(['blue', 'tomato', 'dodgerblue', 'turquoise', 'magenta', 'indigo', 'darkgray']) # since Kmeans clustering with 5     
            markers_ = ['o','s', 'x', 'd', '^']
            markers_cyc =cycle(markers_)  
            
            if ind1 != ind2: 
                plt.figure(figsize=(20,10))
                ax = plt.gca()
                
                for cl_i, kmeans_label_name in zip( range(0, Kmeans_cl_size), kmeans_label_names ): # i: cluster index
                    temp1 = df_traffic_FE_PCA[kmeans.labels_ == cl_i] # subselect by true/false of same length # temp1 still has bs as index
                    temp2 = temp1.loc[ list( set.intersection( set(temp1.index), set(df_topology_sh.index) ) ) ]
                    bs_list_sh = list( temp2.index ) # BS index specific for shortened list & cl_i
                    
                    # df_topology_sh.index is also bs index
                    # loc: by 'index' column
                    
                    plt.scatter( temp2.iloc[:,ind1],temp2.iloc[:,ind2], \
                                c=next(colors), s = s, alpha=alpha , lw=lw, label=kmeans_label_name, \
                                marker = next(markers_cyc) )
                    # plt.scatter(kmeans.cluster_centers_[i, ind1], kmeans.cluster_centers_[i, ind2], marker=markers_[i], s=1000, c=color )
                        # blocked this line. If categorical values, majority vote is better. 
                        
                    if text_disp == 1:     
                        for ii, bb, xx, yy, in zip( range(0,len(bs_list_sh)) , bs_list_sh, \
                                df_traffic_FE_PCA.loc[bs_list_sh].iloc[:, ind1], df_traffic_FE_PCA.loc[bs_list_sh].iloc[:, ind2] ):
                            plt.text(xx+xx_d_text, yy+yy_d_text, str(cl_i)+','+str(bb), fontsize = 12, rotation = 20)      

                plt.legend(loc='upper left', shadow=False, scatterpoints=1, bbox_to_anchor=(1.02, 1), fontsize='large', \
                   numpoints = 15)        
                    # plt.legend(loc='best', shadow=False, scatterpoints=1)
                plt.title( 'After Kmeans clustering of ' + str(num_BS) + ' BS' )  
                plt.xlabel( 'PCA ' + str(ind1+1) + 'st compo' )
                plt.ylabel( 'PCA ' + str(ind2+1) + 'st compo' )
                
                # ax.set_xscale('log')
                # ax.set_yscale('log')
                
                plt.show()
                
    del temp1
    del temp2
    # del aa, bb, cc